# Loading and modifying a SMIRNOFF forcefield

This notebook illustrates how to load a SMIRNOFF forcefield, apply it to an example molecule, get the energy, then manipulate the parameters in the forcefield and update the energy.

## Prep some utility functions/import stuff

In [1]:
# Imports needed
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff.forcefield import ForceField
from openforcefield.utils import get_data_filename
from simtk import openmm, unit
import numpy as np

In [2]:
# Define utility function we'll use to get energy of an OpenMM system
def get_energy(system, positions):
    """
    Return the potential energy.

    Parameters
    ----------
    system : simtk.openmm.System
        The system to check
    positions : simtk.unit.Quantity of dimension (natoms,3) with units of length
        The positions to use
    Returns
    ---------
    energy
    """

    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator)
    context.setPositions(positions)
    state = context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().in_units_of(unit.kilocalories_per_mole)
    return energy

## Example 1: Load a molecule and evaluate its energy before and after a parameter modification

In this example, we load a single ethanol molecule with geometry information, parameterize it using the smirnoff99Frosst forcefield, and evaluate its energy. We then modify the parameter that is applied to the C-O-H angle and re-evaluate the energy.

In [3]:
# Load a molecule
from openforcefield.tests.utils import get_alkethoh_filepath
#molecule = Molecule.from_file(get_alkethoh_filepath('AlkEthOH_c100'))
molecule = Molecule.from_file(get_data_filename('molecules/ethanol.mol2'))

#Get positions for use below
positions = molecule.conformers[0]

# Load forcefield file
ff = ForceField('smirnoff99Frosst.offxml')

# Generate an openforcefield Topology containing only this molecule
topology = molecule.to_topology()

# Create initial system
orig_system = ff.create_openmm_system(topology)

# Get initial energy before parameter modification
orig_energy = get_energy(orig_system, positions)

# Get params for an angle
smirks = '[*:1]-[#8:2]-[*:3]' # SMIRKS for the parameter to retrieve
parameter = ff.get_handler('Angles').parameters[smirks]

# Modify the parameters
parameter.k *= 0.9
parameter.angle *= 1.1

# Evaluate energy after parameter modification
new_system = ff.create_openmm_system(topology)
new_energy = get_energy(new_system, positions)

# Print out energy
print(f"Original energy: {orig_energy}. New energy: {new_energy}")

Original energy: -0.27712854782892227 kcal/mol. New energy: 3.318374972954307 kcal/mol


## Inspect and manipulate parameters

In [4]:
# Create a new ForceField containing the smirnoff99Frosst parameter set:

forcefield = ForceField('smirnoff99Frosst.offxml')

In [5]:
# Create an OpenMM system from a :class:`openforcefield.topology.Topology` object:
molecules = [Molecule.from_smiles('CCO'),
             Molecule.from_smiles('HOH')]
from simtk.openmm import app
pdbfile = app.PDBFile(get_data_filename('systems/packmol_boxes/ethanol_water.pdb'))
top = Topology.from_openmm(pdbfile.topology, unique_molecules=molecules)
orig_system = forcefield.create_openmm_system(top)
orig_energy = get_energy(orig_system, pdbfile.getPositions())

In [6]:

# Modify the long-range van der Waals method to be PME:

forcefield.get_handler('vdW').method = 'PME'

# Inspect the first few vdW parameters:
for vdw_param in forcefield.get_handler('vdW').parameters[0:3]:
    print(vdw_param)

new_system = forcefield.create_openmm_system(top)
new_energy = get_energy(new_system, pdbfile.getPositions())

print(f"Original energy: {orig_energy}. New energy: {new_energy}")

<vdWType with smirks: [#1:1]  epsilon: 0.0157 kcal/mol  id: n1  sigma: 1.069078461768407 A  >
<vdWType with smirks: [#1:1]-[#6X4]  epsilon: 0.0157 kcal/mol  id: n2  sigma: 2.649532787749369 A  >
<vdWType with smirks: [#1:1]-[#6X4]-[#7,#8,#9,#16,#17,#35]  epsilon: 0.0157 kcal/mol  id: n3  sigma: 2.471353044121301 A  >
Original energy: -6216.464064995574 kcal/mol. New energy: -32426.412374521988 kcal/mol


In [19]:
orig_energy = get_energy(orig_system, pdbfile.getPositions())#/10.)
new_energy = get_energy(new_system, pdbfile.getPositions())#/10.)
print(f"Original energy: {orig_energy}. New energy: {new_energy}")

Original energy: 36804319804003.164 kcal/mol. New energy: nan kcal/mol


In [24]:
print(new_system.getDefaultPeriodicBoxVectors())
print(new_system.usesPeriodicBoundaryConditions())

[Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
True
